In [3]:
import numpy as np
import pandas as pd
import scanpy as sc

%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

/home/ing/miniconda3/envs/scrna/lib/python3.7/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [4]:
master_palette = {
    "ACTA2_Fibroblast": "#ffbb78", 
    "APC_Macrophage": "#7f7f7f", 
    "Activated_Bcell": "#c5b0d5", 
    "CDH12_CDH18_Epithelial": "#1f77b4", 
    "DSG3_Epithelial": "#ff7f0e", 
    "Dendritic_cell": "#c49c94", 
    "Endothelial": "#17becf", 
    "FAP_Fibroblast": "#8c564b", 
    "Inflam_Macrophage": "#aec7e8", 
    "Intermediate_Epithelial": "#2ca02c", 
    "KRT_Epithelial": "#9467bd", 
    "Late_Activation_CD8T": "#e377c2", 
    "Naive_Tcell": "#ff9896", 
    "PDGFRB_Fibroblast": "#f7b6d2", 
    "PDPN_Fibroblast": "#98df8a", 
    "Plasma_Bcell": "#c7c7c7", 
    "Proliferating_Epithelial": "#d62728", 
    "Treg": "#dbdb8d", 
    "KRT_13_17_Epithelial": "#bcbd22", 
}
# https://stackoverflow.com/a/29643643
def hex2rgb(h):
    h = h.lstrip('#')
    return tuple(int(h[i:i+2], 16) for i in (0, 2, 4))

print(hex2rgb(master_palette['ACTA2_Fibroblast']))

(255, 187, 120)


In [5]:
adata = sc.read_h5ad("../BladderGEX_run2_working.h5ad")
sc.pp.normalize_total(adata, target_sum=10000)

radata = sc.read_h5ad("../receptor_scoring/receptor_genelists_2020_06_30_scores.h5ad")
radata.obs['SubType_v3'] = adata.obs['SubType_v3']

subtypes = np.unique(radata.obs.SubType_v3)
celltypes = np.unique(radata.obs.BroadCellType_v2)
print(subtypes)
print(celltypes)

radata.shape

['ACTA2_Fibroblast' 'APC_Macrophage' 'Activated_Bcell'
 'CDH12_CDH18_Epithelial' 'DSG3_Epithelial' 'Dendritic_cell' 'Endothelial'
 'FAP_Fibroblast' 'Inflam_Macrophage' 'Intermediate_Epithelial'
 'KRT_Epithelial' 'Late_Activation_CD8T' 'Naive_Tcell' 'PDGFRB_Fibroblast'
 'PDPN_Fibroblast' 'Plasma_Bcell' 'Proliferating_Epithelial' 'Treg']
['Endothelial' 'Epithelial' 'Fibroblast' 'Lymphocyte' 'Myeloid']


(67988, 675)

In [129]:
visium_channels = !cat visium_channels_2.txt
# Add PDCD1LG2_PDCD1
visium_channels.append('PDCD1LG2_PDCD1')
print(len(visium_channels))

1211


In [130]:
ligand_receptor = pd.read_csv("/home/ing/devel/ccsig/data/cabello_aguilar_Ligand_Receptor.csv", index_col=0,header=0)
# Add PDCD1LG2_PDCD1
ligand_receptor.loc[ligand_receptor.index[-1]+1, :] = ['PDCD1LG2', 'PDCD1']
ligands = np.unique(ligand_receptor.ligand.tolist())
print(len(ligands))
ligand_receptor

807


,ligand,receptor
1,A2M,LRP1
2,AANAT,MTNR1A
3,AANAT,MTNR1B
4,ACE,AGTR2
5,ACE,BDKRB2
...,...,...
3248,WNT2B,FZD4
3249,CCL18,CCR8
3250,CD52,SIGLEC10
3251,CD200,CD200R1


In [141]:
percent_cutoff = 0.05

diffrx = {}
for ct in celltypes:
# for ct in ['Fibroblast']:
    if ct == 'Endothelial': continue
    rd = radata[radata.obs.BroadCellType_v2 == ct]
    sc.tl.rank_genes_groups(rd, groupby='SubType_v3', method='wilcoxon')
    ct_subtypes = np.unique(radata.obs.SubType_v3.values[radata.obs.BroadCellType_v2 == ct])
    print(list(ct_subtypes))
    rdx = pd.DataFrame(rd.X.toarray() > 0, index=rd.obs_names, columns=rd.var_names)
    rdx['subtype'] = rd.obs.SubType_v3
    rdx = rdx.groupby('subtype').mean()

    for st in ct_subtypes:
        expressed_receptors = rdx.columns[rdx.loc[st] > percent_cutoff].tolist() # Percent expression cutoff
        df = sc.get.rank_genes_groups_df(rd, st)
        df = df.query("logfoldchanges > 0.25 & pvals_adj < 0.05")
#         df = df.query("logfoldchanges > 0.5")
        df = df.loc[df.names.isin(expressed_receptors)]
        df['subtype'] = [st] * df.shape[0]
        diffrx[st] = df.copy()
        
rd = radata[radata.obs.BroadCellType_v2.isin(['Endothelial', 'Fibroblast'])]
sc.tl.rank_genes_groups(rd, groupby='SubType_v3', method='wilcoxon')
df = sc.get.rank_genes_groups_df(rd,'Endothelial')
df = df.query("logfoldchanges > 0.25")
df['subtype'] = ['Endothelial'] * df.shape[0]
diffrx['Endothelial'] = df.copy()

Trying to set attribute `.uns` of view, copying.


['CDH12_CDH18_Epithelial', 'DSG3_Epithelial', 'Intermediate_Epithelial', 'KRT_Epithelial', 'Proliferating_Epithelial']


/home/ing/miniconda3/envs/scrna/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
Trying to set attribute `.uns` of view, copying.
/home/ing/miniconda3/envs/scrna/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
Trying to set attribute `.uns` of view, copying.
/home/ing/miniconda3/envs/scrna/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
Trying to set attribute `.uns` of view, copying.


['ACTA2_Fibroblast', 'FAP_Fibroblast', 'PDGFRB_Fibroblast', 'PDPN_Fibroblast']
['Activated_Bcell', 'Late_Activation_CD8T', 'Naive_Tcell', 'Plasma_Bcell', 'Treg']


/home/ing/miniconda3/envs/scrna/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
Trying to set attribute `.uns` of view, copying.


['APC_Macrophage', 'Dendritic_cell', 'Inflam_Macrophage']


In [142]:
sender = 'CDH12_CDH18_Epithelial'
background_set = ['Epithelial'] # Background cells to use for ligand enrichment

send_ad = adata[adata.obs.BroadCellType_v2.isin(background_set), adata.var_names.isin(ligands)]
sc.tl.rank_genes_groups(send_ad, groupby='SubType_v3', method='wilcoxon', n_genes=send_ad.shape[1])
sender_df = sc.get.rank_genes_groups_df(send_ad, sender)
sender_df = sender_df.query("logfoldchanges > 0.5 & pvals_adj < 0.05")
sender_ligands = sender_df.names.tolist()

N=25

# receivers = ['CDH12_CDH18_Epithelial', 'DSG3_Epithelial', 'Intermediate_Epithelial', 
#              'KRT_Epithelial', 'Proliferating_Epithelial']
# receivers = ['ACTA2_Fibroblast', 'FAP_Fibroblast', 'PDGFRB_Fibroblast', 'PDPN_Fibroblast'] 
# receivers = ['Endothelial']
receivers = ['Late_Activation_CD8T', 'Naive_Tcell', 'Treg']
# receivers = ['Activated_Bcell', 'Plasma_Bcell']
# receivers += ['Activated_Bcell', 'Late_Activation_CD8T', 'Naive_Tcell', 'Plasma_Bcell', 'Treg']
# receivers = ['APC_Macrophage', 'Dendritic_cell', 'Inflam_Macrophage']

# Track the matched ligand-receptors 
agreement = {}
agreement_n = {}
for receiver in receivers:
    receptor_set = diffrx[receiver].names.values[:N]
    receptor_set = diffrx[receiver].names.values
    matched = ligand_receptor.loc[ligand_receptor.receptor.isin( receptor_set )]
    ligand_set = np.unique(matched['ligand'].values)

    lx = adata[:, adata.var_names.isin(ligand_set)]

    dx = pd.DataFrame(lx.X.toarray() > 0, index=adata.obs_names, columns=lx.var_names)
    dx['SubType_v3'] = adata.obs.SubType_v3
    print(dx.shape)

    pct = dx.groupby('SubType_v3').mean() > percent_cutoff

    hits = pct.columns[pct.loc[sender]].tolist()
    hits = [h for h in hits if h in sender_ligands]

    channels = []
    for h in hits:
        rr = matched.loc[matched.ligand == h].receptor.values
        for r in rr:
            if r in receptor_set:
                channels.append(f'{h}_{r}')
                
    channels = sorted([c for c in channels if c in visium_channels])
    
    agreement[receiver] = channels
    agreement_n[receiver] = len(channels)
    print(sender, '-->', receiver, channels, len(channels))

# sender_agreement[sender] = agreement

/home/ing/miniconda3/envs/scrna/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):
/home/ing/miniconda3/envs/scrna/lib/python3.7/site-packages/anndata/_core/anndata.py:1192: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_string_dtype(df[key]) and not is_categorical(df[key])
Trying to set attribute `.uns` of view, copying.
/home/ing/miniconda3/envs/scrna/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


(67988, 151)
CDH12_CDH18_Epithelial --> Late_Activation_CD8T ['C5_C5AR1', 'COL4A3_ITGA1', 'COL4A4_ITGA1', 'COL5A1_ITGA1', 'COL5A2_ITGA1', 'COL6A3_ITGA1', 'COL8A1_ITGA1', 'IL34_CSF1R', 'LAMA1_ITGA1', 'LAMA2_ITGA1', 'OMG_TNFRSF1B', 'PDCD1LG2_PDCD1', 'PECAM1_CD38', 'PTDSS1_JMJD6', 'REN_ATP6AP2', 'SORBS1_ITGA1', 'TGFB1_TGFBR3', 'TGFB2_TGFBR3', 'TNC_CNTN1', 'VEGFD_ITGA1', 'WNT3A_ATP6AP2'] 21


/home/ing/miniconda3/envs/scrna/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


(67988, 52)
CDH12_CDH18_Epithelial --> Naive_Tcell ['IL16_CD4'] 1


/home/ing/miniconda3/envs/scrna/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


(67988, 132)
CDH12_CDH18_Epithelial --> Treg ['ADAM12_SDC4', 'BMP6_ACVR1', 'BTC_ERBB3', 'EGF_ERBB3', 'IL16_CD4', 'LAMA1_SDC4', 'NRG1_ERBB3', 'NRG2_ERBB3', 'OMG_TNFRSF1B', 'RSPO3_SDC4', 'TGFB2_ACVR1', 'TNC_SDC4'] 12


In [143]:
all_ligands = set()
all_receptors = set()
for k, vv in agreement.items():
    for v in vv:
        ligand, receptor = v.split('_')
        all_ligands = all_ligands.union(set([ligand]))
        all_receptors = all_receptors.union(set([receptor]))
        
sd = adata[adata.obs.BroadCellType_v2.isin(background_set), adata.var_names.isin(list(all_ligands))]
# Percent of cells with nonzero ligand expression in sender
sdx = pd.DataFrame(sd.X.toarray() > 0, index=sd.obs_names, columns=sd.var_names)
sdx['subtype'] = sd.obs.SubType_v3
sdx = sdx.groupby('subtype').mean().loc[[sender]]

# Mean ligand expression in sender 
sdxe = pd.DataFrame(sd.X.toarray(), index=sd.obs_names, columns=sd.var_names)
sdxe['subtype'] = sd.obs.SubType_v3
sdxe = sdxe.groupby('subtype').mean().loc[[sender]]
print(sdx.shape)

rd = radata[:, radata.var_names.isin(list(all_receptors))]
# Percent of cells with non-zero receptor expression
rdx = pd.DataFrame(rd.X.toarray() > 0, index=rd.obs_names, columns=rd.var_names)
rdx['subtype'] = rd.obs.SubType_v3
rdx = rdx.groupby('subtype').mean()
rdx = rdx.loc[receivers, :]

# Mean receptor score
rdxe = pd.DataFrame(rd.X.toarray(), index=rd.obs_names, columns=rd.var_names)
rdxe['subtype'] = rd.obs.SubType_v3
rdxe = rdxe.groupby('subtype').mean()
rdxe = rdxe.loc[receivers, :]
print(rdx.shape)

/home/ing/miniconda3/envs/scrna/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


(1, 29)
(3, 14)


In [144]:
receiver_celltype = 'Tcell'
outbase = f'01_Circos_interactions/{sender}__to__{receiver_celltype}_2'

In [145]:
import seaborn as sns
reds = [tuple(int(ch * 255) for ch in c) for c in sns.color_palette('Reds', 10)]
blues = [tuple(int(ch * 255) for ch in c) for c in sns.color_palette('Blues', 10)]
print(reds)
# sns.palplot(reds)
# sns.palplot(blues)

[(254, 229, 218), (253, 207, 188), (252, 180, 153), (252, 149, 117), (251, 120, 88), (246, 89, 63), (236, 56, 42), (208, 29, 31), (181, 18, 24), (147, 10, 18)]


In [146]:
def get_receptor_color(rdxe, receptor, receiver, cmap):
    # Change color scale
#     max_val = np.max(rdxe.loc[:, receptor].values)
    max_val = np.max(rdxe.values)
    bins = np.linspace(0, max_val, 10)
    b = np.digitize(rdxe.loc[receiver, receptor], bins, right=True)
    color = cmap[b]
    color = f','.join([f'{c}' for c in color])
    return color
    
# print(get_receptor_color(rdxe, 'ABCA1', 'ACTA2_Fibroblast', blues))
    
    
def write_receptor_karyotype(f, hlf, txtf, start, semi_circle, cmap=blues):
    # Also keep track of where to place ligands
    ligand_order = []
    
    # And coordinates for the links
    receptor_coords = {}
    
    total_receptors_with_repeats = 0
    for receiver, channels in agreement.items():
        receiver_receptors = set([x.split('_')[1] for x in channels])
        total_receptors_with_repeats += len(receiver_receptors)

    print(total_receptors_with_repeats)
    area_per_receptor = int(semi_circle / total_receptors_with_repeats)
    print(area_per_receptor)

    celltype_start = start
    receptor_start = start
    for receiver, channels in agreement.items():
        receptor_coords[receiver] = {}
        
        receiver_receptors = [x.split('_')[1] for x in channels]
        ligs = [x.split('_')[0] for x in channels]
        printed_receptors = []
#         for receptor, ligand in zip(receiver_receptors, ligs):
        for ch in channels:
            ligand, receptor = ch.split('_')
            if ligand not in ligand_order:
                ligand_order.append(ligand)
                
            if receptor in printed_receptors:
                continue
            else:
                printed_receptors.append(receptor)
                
            receptor_end = receptor_start + area_per_receptor
            
            # Track the receptor coordinates on this receiver
            receptor_coords[receiver][receptor] = (receptor_start, receptor_end)
            
            color = get_receptor_color(rdxe, receptor, receiver, cmap)
            
            hl = f'{receiver} {receptor_start} {receptor_end} fill_color={color}\n'
            hlf.write(hl)
            
            txt = f'{receiver} {receptor_start} {receptor_end} {receptor} color={color}\n'
            txtf.write(txt)
            
            receptor_start = receptor_end
        # The value of receptor start is now where our current cell type should stop
        color = ','.join([f'{v}' for v in hex2rgb(master_palette[receiver])])
        celltype_line = f'chr - {receiver} {receiver} {celltype_start} {receptor_start} {color}\n'
        f.write(celltype_line)
        # start for the next cell type
        celltype_start = receptor_start
        
    return ligand_order, receptor_coords

In [147]:
def get_ligand_color(sdxe, ligand, sender, cmap):
    # Change color scale
#     max_val = np.max(rdxe.loc[:, receptor].values)
    max_val = np.max(sdxe.values)
    bins = np.linspace(0, max_val, 10)
    b = np.digitize(sdxe.loc[sender, ligand], bins, right=True)
    color = cmap[b]
    color = f','.join([f'{c}' for c in color])
    return color

# Find and place ligands 
def write_ligands(hlf, txtf, start, total_ticks, ligand_order, cmap=reds):
    # Track ligand positions on the sender
    ligand_coords = {}
    
    n_ligands = len(ligand_order)
    print(n_ligands)
    borders = np.linspace(start, total_ticks, n_ligands+1, dtype=int)
    # Go in the reverse order that we wrote the receptors:
    for i, ligand in enumerate(ligand_order[::-1]):
        ligand_start = borders[i]
        ligand_end = borders[i+1]
        color = get_ligand_color(sdxe, ligand, sender, reds)
        hl = f'{sender} {ligand_start} {ligand_end} fill_color={color}\n'
        txt = f'{sender} {ligand_start} {ligand_end} {ligand} color={color}\n'
        
        ligand_coords[ligand] = (ligand_start, ligand_end)

        hlf.write(hl)
        txtf.write(txt)
    
    return ligand_coords

In [148]:
# 0 = fully opaque
MAX_TRANSPARENCY=127

def draw_links(linkf, sdxe, rdxe, agreement, ligand_coords, receptor_coords, receiver_order=None):
    if receiver_order is None:
        receiver_order = list(agreemen.keys())
#     for receiver, channels in agreement.items():

    max_interaction = 0
    for receiver in receiver_order:
        channels = agreement[receiver]
        for ch in channels:
            ligand, receptor = ch.split('_')
            rexp = rdxe.loc[receiver, receptor]
            sexp = sdxe.loc[sender, ligand]
            i = rexp * sexp
            if i > max_interaction:
                max_interaction = i
            
    print(max_interaction) 
    for receiver in receiver_order:
        channels = agreement[receiver]
        
        color = ','.join([f'{v}' for v in hex2rgb(master_palette[receiver])])
        
        for ch in channels:
            ligand, receptor = ch.split('_')
            l_c = ligand_coords[ligand]
            r_c = receptor_coords[receiver][receptor]
            
            rexp = rdxe.loc[receiver, receptor]
            sexp = sdxe.loc[sender, ligand]
            i = rexp * sexp
#             trx = MAX_TRANSPARENCY - int(MAX_TRANSPARENCY * (i / max_interaction))
            trx = f'{min(1-(i / max_interaction)**2, 0.9):3.3f}'
            print(sender, ligand, receiver, receptor, i, trx)
#             spct = sdx.loc[sender, ligand]
#             hl = (l_c[1] - l_c[0]) / 2
#             lm = l_c[0] + ((l_c[1] - l_c[0]) / 2)
#             l0 = int(lm - hl*spct)
#             l1 = int(lm + hl*spct)
            l0 = l_c[0]
            l1 = l_c[1]
            
#             rpct = rdx.loc[receiver, receptor]
#             hr = (r_c[1] - r_c[0]) / 2
#             rm = r_c[0] + ((r_c[1] - r_c[0]) / 2)
#             r0 = int(rm - hr*rpct)
#             r1 = int(rm + hr*rpct)
            r0 = r_c[0]
            r1 = r_c[1]
            
            
            link = f'{sender} {l0} {l1} {receiver} {r0} {r1} color={color},{trx}\n'
            
            linkf.write(link)

In [149]:
outbase

'01_Circos_interactions/CDH12_CDH18_Epithelial__to__Tcell_2'

In [150]:
'''
karyotype.txt

chr - [name] [name] start stop R,G,B 
# start stop in int
# R,G,B in uint8

> 
'''
total_ticks = 10000
semi_circle = int(total_ticks/2)
print(semi_circle)

total_celltypes = len(sdx.index) + len(rdx.index)
print(total_celltypes)

f = open(f'{outbase}/karyotype.txt', 'w+')
hlf = open(f'{outbase}/highlights.txt', 'w+')
txtf = open(f'{outbase}/genes.txt', 'w+')
linkf = open(f'{outbase}/links.txt', 'w+')

# Circos starts at 12 o'clock
ligand_order, receptor_coords = write_receptor_karyotype(f, hlf, txtf, 0, semi_circle)

# write the sending semi-circle
color = ','.join([f'{v}' for v in hex2rgb(master_palette[sender])])
line = f'chr - {sender} {sender} {semi_circle} {total_ticks} {color}\n'
print(line)
f.write(line)

# now write the sending ligands
ligand_coords = write_ligands(hlf, txtf, semi_circle, total_ticks, ligand_order)

receiver_order = list(agreement.keys())[::-1]
draw_links(linkf, sdxe, rdxe, agreement, ligand_coords, receptor_coords, receiver_order=receiver_order)

f.close()
hlf.close()
txtf.close()
linkf.close()

5000
4
16
312
chr - CDH12_CDH18_Epithelial CDH12_CDH18_Epithelial 5000 10000 31,119,180

29
0.20970494
CDH12_CDH18_Epithelial ADAM12 Treg SDC4 0.07445297 0.874
CDH12_CDH18_Epithelial BMP6 Treg ACVR1 0.037900053 0.900
CDH12_CDH18_Epithelial BTC Treg ERBB3 0.005848309 0.900
CDH12_CDH18_Epithelial EGF Treg ERBB3 0.010261406 0.900
CDH12_CDH18_Epithelial IL16 Treg CD4 0.02157155 0.900
CDH12_CDH18_Epithelial LAMA1 Treg SDC4 0.058682818 0.900
CDH12_CDH18_Epithelial NRG1 Treg ERBB3 0.11031577 0.723
CDH12_CDH18_Epithelial NRG2 Treg ERBB3 0.013527654 0.900
CDH12_CDH18_Epithelial OMG Treg TNFRSF1B 0.027113637 0.900
CDH12_CDH18_Epithelial RSPO3 Treg SDC4 0.0120543 0.900
CDH12_CDH18_Epithelial TGFB2 Treg ACVR1 0.03464693 0.900
CDH12_CDH18_Epithelial TNC Treg SDC4 0.031916622 0.900
CDH12_CDH18_Epithelial IL16 Naive_Tcell CD4 0.012561643 0.900
CDH12_CDH18_Epithelial C5 Late_Activation_CD8T C5AR1 0.007795845 0.900
CDH12_CDH18_Epithelial COL4A3 Late_Activation_CD8T ITGA1 0.0668966 0.898
CDH12_CDH18_Epi